### LLMChains & OutputParsers

Instead of just using models you can combine a model and a prompt. This can be done with the LLMChain class.
We will additional, more complex chains in this notebook

In [16]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [26]:
TEMPLATE = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment? Sentiment is required.
subject: What subject is the text about? Use exactly one word. Use 'None' if no subject was provided.
price: How much did the customer pay? Use 'None' if no price was provided.

Format the output as JSON with the following keys:
sentiment
subject
price

text: {input}
"""

In [27]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.llm import LLMChain

llm = ChatOpenAI(model="gpt-4o-mini")

prompt_template = ChatPromptTemplate.from_template(template=TEMPLATE)
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.invoke(input="I ordered pizza salami from the restaurant Bellavista. It was ok, but the dough could have been a bit more crisp.")

{'input': 'I ordered pizza salami from the restaurant Bellavista. It was ok, but the dough could have been a bit more crisp.',
 'text': '```json\n{\n  "sentiment": "neutral",\n  "subject": "pizza",\n  "price": "None"\n}\n```'}

### Response Schemas

There were two issues with the output: The output also contains text and the output is just a string, not a dictionary.

In [28]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

response_schemas = [
    ResponseSchema(name="sentiment", description="is the text in a positive, neutral or negative sentiment? Sentiment is required."),
    ResponseSchema(name="subject", description="What subject is the text about? Use exactly one word. Use None if no price was provided."),
    ResponseSchema(name="price", description="How much did the customer pay? Use None if no price was provided.", type="float") # type can be string, float, int, bool, list, dict, etc.
]
print(response_schemas)

[ResponseSchema(name='sentiment', description='is the text in a positive, neutral or negative sentiment? Sentiment is required.', type='string'), ResponseSchema(name='subject', description='What subject is the text about? Use exactly one word. Use None if no price was provided.', type='string'), ResponseSchema(name='price', description='How much did the customer pay? Use None if no price was provided.', type='float')]


In [29]:
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"sentiment": string  // is the text in a positive, neutral or negative sentiment? Sentiment is required.
	"subject": string  // What subject is the text about? Use exactly one word. Use None if no price was provided.
	"price": float  // How much did the customer pay? Use None if no price was provided.
}
```


In [30]:
# Create prompt template
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate

prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "Interprete the text and evaluate the text. "
            "sentiment: is the text in a positive, neutral or negative sentiment? "
            "subject: What subject is the text about? Use exactly one word. "
            "Just return the JSON, do not add ANYTHING, NO INTERPRETATION! "
            "text: {input}\n"
            "{format_instructions}\n"
        )
    ],
    input_variables=["input"],
    partial_variables={"format_instructions": format_instructions} # Useful when you want to use the same prompt for different output formats
)

In [42]:
_input = prompt.format_prompt(input="I ordered pizza salami from the restaurant Bellavista. It was ok, but the dough could have been a bit more crisp.")
print(_input)

messages=[SystemMessage(content='Interprete the text and evaluate the text. sentiment: is the text in a positive, neutral or negative sentiment? subject: What subject is the text about? Use exactly one word. Just return the JSON, do not add ANYTHING, NO INTERPRETATION! text: I ordered pizza salami from the restaurant Bellavista. It was ok, but the dough could have been a bit more crisp.\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"sentiment": string  // is the text in a positive, neutral or negative sentiment? Sentiment is required.\n\t"subject": string  // What subject is the text about? Use exactly one word. Use None if no price was provided.\n\t"price": float  // How much did the customer pay? Use None if no price was provided.\n}\n```\n', additional_kwargs={}, response_metadata={})]


In [ ]:
"""
messages = [
    SystemMessage(
        content='Interprete the text and evaluate the text. sentiment: is the text in a positive, neutral or negative sentiment? '
        'subject: What subject is the text about? Use exactly one word. Just return the JSON, do not add ANYTHING, NO INTERPRETATION! '
        'text: I ordered pizza salami from the restaurant Bellavista. It was ok, but the dough could have been a bit more crisp.\n'
        'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n'
        '```json\n'
        '{\n'
        '\t"sentiment": string  // is the text in a positive, neutral or negative sentiment? Sentiment is required.\n'
        '\t"subject": string  // What subject is the text about? Use exactly one word. Use None if no price was provided.\n'
        '\t"price": float  // How much did the customer pay? Use None if no price was provided.\n'
        '}\n'
        '```\n',
        additional_kwargs={}, # It's commonly used to store extra information that might be specific to certain LLM providers or custom implementations
        response_metadata={} # Token usage etc.
    )
]
"""

In [53]:
output = llm.invoke(_input.to_messages()) # to_messages() converts the input to a list of messages. If the list has more than one message, the LLM will be called multiple times.
print(output.content)

```json
{
	"sentiment": "neutral",
	"subject": "pizza",
	"price": null
}
```


In [36]:
json_output = parser.parse(output.content)
print(json_output)

{'sentiment': 'neutral', 'subject': 'pizza', 'price': None}


In [37]:
json_output.get("sentiment")

'neutral'